In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#%load_ext autotime  # must be added to dependencies
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from structure_factor.structure_factor_new import StructureFactor
from structure_factor.spatstat_interface import SpatstatInterface
#from structure_factor.spatial_windows import UnitBoxWindow


# test the method  ``compute_scattering_intensity`` of the calss ``StructureFactor``

## loading data for test :
Poisson point process (data_poisson_disk, data_poisson_cube), Ginibre point pross (data_ginibre), lattice $\mathbb{Z}^2$ (data_z_2) and matching processus of Michael Andreas Klatt, Günter Last, D. Yogeshwaran that we will denoted by kly defined in https://arxiv.org/abs/1810.00265 (data_kly)

We load big set of data that we already sampled for time perpose. As we know that the method ``compute_scattering_intensity``work only for point configurations in **cubic windows** so for a point processes lying in a random window we will only take the points falling in a cubic window .
For example a configuration of the Ginibre point process is always contained in a ball, so we will project it to a cubic window.
To compare the effect of **dismissing points** we will test the poisson point process ``data_poisson_disk`` sampled in a disk 
then projected to a cubic window ``data_poisson_cube_2`` and a poisson point process ``data_poisson_cube`` sampled directely in a cubic window. Note that you can creat a Poisson point process directly using the method ``homogeneous_poisson_process``and ``spatial_window``, an example will be given in the end of this notebook.

In [ ]:
import pickle
path_data = "../data"
with open(os.path.join(path_data, "test_data.dat"), "rb") as data:
    data_poisson_cube, L_poisson, data_poisson_disk, radius_poisson, data_ginibre, radius_ginibre, data_kly, L_kly, data_z_2, L_z_2 = pickle.load(data, encoding="bytes")

In [ ]:
print("data_poisson_disk.shape", data_poisson_disk.shape, "radius_poisson", radius_poisson)
print("data_poisson_cube.shape", data_poisson_cube.shape, "L_poisson", L_poisson )
print("data_ginibre.shape", data_ginibre.shape, "radius_ginibre", radius_ginibre )
print("data_kly.shape", data_kly.shape, "L_kly", L_kly)
print("data_z_2.shape", data_z_2.shape, "L_z_2", L_z_2)

## Please consider using code from `homogeneous_poisson_process.py` and `spatial_windows.py` as presented in `Poisson-process-and-windows.ipynb`

In [ ]:
# truncating data_poisson and Ginibre in a cubic window
# poisson, and ginibre are contained in a ball while klatt and z_2 are in a rectangular window L = 300
L_poisson_2 = np.floor(radius_poisson/np.sqrt(2))
index_poisson_in_cube = np.logical_and((np.abs(data_poisson_disk[:,0]) < (L_poisson_2/2)) , (np.abs(data_poisson_disk[:,1]) < (L_poisson_2/2)))
data_poisson_cube_2 = data_poisson_disk[index_poisson_in_cube]


L_ginibre = radius_ginibre/np.sqrt(2)
index_ginibre_in_cube = np.logical_and((np.abs(data_ginibre[:,0]) < L_ginibre/2) , (np.abs(data_ginibre[:,1]) < L_ginibre/2))
data_ginibre_cube = data_ginibre[index_ginibre_in_cube]

print("data_poisson_cube.shape", data_poisson_cube_2.shape)
print("data_ginibre_cube.shape", data_ginibre.shape )
fig, ax = plt.subplots(2, 2, figsize=(9,6))
ax[0,0].plot(data_ginibre[:,0], data_ginibre[:,1], 'b,')
ax[0,0].title.set_text("data ginibre")
ax[0,1].plot(data_ginibre_cube[:,0], data_ginibre_cube[:,1], 'b,')
ax[0,1].title.set_text("data ginibre truncated in a cubic window")
ax[1,0].plot(data_poisson_disk[:,0], data_poisson_disk[:,1], 'b,')
ax[1,0].title.set_text("data poisson")
ax[1,1].plot(data_poisson_cube_2[:,0], data_poisson_cube_2[:,1], 'b,')
ax[1,1].title.set_text("data poisson truncated in a cubic window")

In [ ]:
sf_poisson_cube = StructureFactor(data_poisson_cube, intensity=1 )
sf_poisson_cube_2 = StructureFactor(data_poisson_cube_2, intensity=1 )
sf_ginibre = StructureFactor(data_ginibre, intensity=1/np.pi )
sf_ginibre_cube = StructureFactor(data_ginibre_cube, intensity = 1/np.pi )
sf_kly = StructureFactor(data_kly, intensity= 1)
sf_z_2 = StructureFactor(data_z_2, intensity=1)

# for Poisson point process
we know that the pair correlation function and the structure factor of the Poisson point process are equal to 1 so we always plot the line  $𝑦=1$  representing the théoretical values for a Poisson point process

In [ ]:
wave_length_poisson, scattering_intensity_poisson = sf_poisson_cube.compute_scattering_intensity(L=L_poisson,maximum_k =10,meshgrid_size=None)

the method ``plot_scattering_intensity`` take as input the output of the method ``compute_scattering_intensity`` to plot them.
also take as optional argument ``binning_parameter``correspnding to the parameters used for binning of ``si``, and ``exact_sf`` which the true sctructure factor function

In [ ]:
sf_poisson_cube.plot_scattering_intensity(wave_length_poisson, scattering_intensity_poisson, bins=40)

In [ ]:
wave_length_poisson, scattering_intensity_poisson = sf_poisson_cube.compute_scattering_intensity(L=L_poisson,maximum_k =10,meshgrid_size=20)

In [ ]:
sf_poisson_cube.plot_scattering_intensity(wave_length_poisson, scattering_intensity_poisson, plot_type="all",  bins=40, )

In [ ]:
wave_length_poisson_2, scattering_intensity_poisson_2 = sf_poisson_cube_2.compute_scattering_intensity(L=L_poisson_2,maximum_k =10,meshgrid_size=150)

In [ ]:
sf_poisson_cube_2.plot_scattering_intensity(wave_length_poisson_2, scattering_intensity_poisson_2, plot_type="all",  bins=40 )

# for Ginibre point process

In [ ]:
exact_pcf_ginibre = lambda x : 1 - np.exp(-x**2)
exact_sf_ginibre = lambda x : 1 - np.exp(-x**2/4) # exact structure factor for the Ginibre point process

In [ ]:
x=np.linspace(0,10, 1000)
plt.plot(x, exact_sf_ginibre(x), 'r')
plt.title("sf of Ginibre")
plt.show()

In [ ]:
norm_k_ginibre, scattering_intensity_ginibre = sf_ginibre_cube.compute_scattering_intensity(L=L_ginibre,maximum_k =20)


In [ ]:
sf_ginibre_cube.plot_scattering_intensity(norm_k_ginibre, scattering_intensity_ginibre, plot_type="plot", exact_sf= exact_sf_ginibre, bins=100 )

In [ ]:
x=np.linspace(0,25, 1000)
plt.plot(x, exact_sf_ginibre(x), 'r', label="exat sf ginibre ")
plt.plot(norm_k_ginibre, scattering_intensity_ginibre, 'k.')
plt.show()

In [ ]:
norm_k_ginibre_, scattering_intensity_ginibre_ = sf_ginibre_cube.compute_scattering_intensity(L=L_ginibre,maximum_k =10,meshgrid_size=300)

In [ ]:
sf_ginibre_cube.plot_scattering_intensity(norm_k_ginibre_, scattering_intensity_ginibre_, plot_type="all", bins=40 )

# for the processus of  Michael Andreas Klatt, Günter Last, D. Yogeshwaran that we will denoted by kly defined in https://arxiv.org/abs/1810.00265

In [ ]:
norm_k_kly, si_kly = sf_kly.compute_scattering_intensity(L=300, maximum_k=20)

In [ ]:
sf_kly.plot_scattering_intensity(norm_k_kly, si_kly, plot_type="plot", bins=40 )

In [ ]:
norm_k_kly_, si_kly_ = sf_kly.compute_scattering_intensity(L=300, maximum_k=10, meshgrid_size=150)

In [ ]:
sf_kly.plot_scattering_intensity(norm_k_kly_, si_kly_, plot_type="all", bins=40)

# for a lattice $\mathbb{Z}^2$

In [ ]:
norm_k_z2, si_z2 = sf_z_2.compute_scattering_intensity(L=300, maximum_k=50)

In [ ]:
sf_z_2.plot_scattering_intensity(norm_k_z2, si_z2, plot_type="plot", bins=40 )

In [ ]:
plt.plot(norm_k_z2, si_z2, 'b.')

In [ ]:
norm_k_z2_, si_z2_ = sf_z_2.compute_scattering_intensity(L=300, maximum_k=20, meshgrid_size=150)

In [ ]:
sf_z_2.plot_scattering_intensity(norm_k_z2_, si_z2_, plot_type="all", bins=40)

# testing ``compute_structure_factor_via_hankel``

the method ``compute_structure_factor_via_hankel`` take in its argument *pcf* which is the pair correlation function $g$. If the pair correlation function is unkown, a discreat approximation on a vector *r* could be held using the method ``compute_pcf`` then the output discret sample $g(r)$ must be interpolated before passing it to ``compute_structure_factor_via_hankel`` since its argument *pcf* is an object of type **function**. This can be done using the method ``interpolate_pair_correlation_function``.

In [ ]:
sf_poisson_disk = StructureFactor(data_poisson_disk, intensity=1 )
pcf_ppp_poisson = sf_poisson_disk.compute_pcf(radius=100, method="ppp", correction="best")
pcf_ppp_poisson

In [ ]:
 sf_poisson_disk.plot_pcf(pcf_ppp_poisson, figsize=(8,5))

In [ ]:

interval, interpolated_pcf_ppp_poisson = sf_poisson_disk.interpolate_pcf(r=pcf_ppp_poisson["r"],
                                                               pcf_r=pcf_ppp_poisson["iso"],
                                                               clean=True ) 



In [ ]:
interval["r_max"]


In [ ]:
k_= np.linspace(3,10, 100)
sf_poisson_ogata = sf_poisson_disk.compute_structure_factor_via_hankel(interpolated_pcf_ppp_poisson, k =k_,
                                                                       method="Ogata", r_max= interval["r_max"])

In [ ]:
print(sf_poisson_disk.k_min)
print((3.5 * np.pi) / (interval["r_max"] *0.1))

In [ ]:
plt.plot( k_ , sf_poisson_ogata, 'b.')

In [ ]:
k_= np.linspace(1,10, 1000)
sf_poisson_baddour = sf_poisson_disk.compute_structure_factor_via_hankel(interpolated_pcf_ppp_poisson, k =k_, 
                                                                       method ="BaddourChouinard",
                                                                       r_max=17, nb_points=100)

In [ ]:
plt.plot( k_ , sf_poisson_baddour, 'b.')

In [ ]:
r_vect = np.linspace(0,50, 500)
pcf_fv_poisson = sf_poisson_disk.compute_pcf(radius=200, method="fv")
pcf_fv_poisson

In [ ]:
r_vect = np.linspace(0,50, 500)
pcf_fv_poisson = sf_poisson_disk.compute_pcf(radius=200, method="fv", Kest=dict(r=r_vect), fv=dict(spar=0.1,  method="a"))
pcf_fv_poisson

In [ ]:
 sf_poisson_disk.plot_pcf(pcf_fv_poisson, figsize=(8,5))

In [ ]:
exact_pcf_ginibre = lambda x : 1 - np.exp(-x**2)
exact_sf_ginibre = lambda x : 1 - np.exp(-x**2/4)

In [ ]:
k_= np.linspace(0,10, 1000)
k, sf_ginibre_baddour = sf_ginibre.compute_structure_factor_via_hankel(exact_pcf_ginibre, k =k_, 
                                                                       method ="BaddourChouinard",
                                                                       r_max=17, nb_points=100)
plt.plot(k_ , sf_ginibre_baddour, 'b.')
plt.plot(k_, exact_sf_ginibre(k_), 'r')
plt.show

In [ ]:
k_= np.linspace(1,10, 1000)
k, sf_ginibre_baddour = sf_ginibre.compute_structure_factor_via_hankel(exact_pcf_ginibre, k =k_, 
                                                                       method ="BaddourChouinard",
                                                                       r_max=17, nb_points=100)
plt.plot(k_ , sf_ginibre_baddour, 'b.')
plt.plot(k_, exact_sf_ginibre(k_), 'r')
plt.show

In [ ]:
sf_ginibre_baddour.shape

In [ ]:
k_, sf_ginibre_baddour = sf_ginibre.compute_structure_factor_via_hankel(exact_pcf_ginibre, 
                                                                       method ="BaddourChouinard",
                                                                       r_max=17, nb_points=100)
plt.plot(k_ , sf_ginibre_baddour, 'b.')
plt.plot(k_, exact_sf_ginibre(k_), 'r')
plt.show

In [ ]:
k= np.linspace(0,10, 1000)
k, sf_ginibre_ogata = sf_ginibre.compute_structure_factor_via_hankel(exact_pcf_ginibre, k =k, 
                                                                       method ="Ogata", nb_points=1000)
plt.plot(k , sf_ginibre_ogata, 'b.')
plt.plot(k, exact_sf_ginibre(k), 'r')
plt.show